## Mistral 7B

In [8]:
# Quantization configuration using bitsandbytes library
compute_dtype = getattr(torch, "bfloat16")  # Set computation data type to bfloat16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # Enable loading the model in 4-bit precision
    bnb_4bit_quant_type="nf4",            # Specify quantization type as Normal Float 4
    bnb_4bit_compute_dtype=compute_dtype, # Set computation data type
    bnb_4bit_use_double_quant=True,       # Use double quantization for better accuracy
)

# Load the pre-trained model with the specified quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Apply quantization configuration
    device_map="auto"                # Automatically map layers to devices
)

# Prepare the model for k-bit (e.g., 4-bit) training
model = prepare_model_for_kbit_training(model)
model.config.pad_token_id = tokenizer.pad_token_id  # Set the model's padding token ID

loading configuration file config.json from cache at /home/cata/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.3/snapshots/d8cadc02ac76bd617a919d50b092e59d2d110aff/config.json
Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-v0.3",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 32768
}

CUDA backend validation successful.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bit

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [4]:
def format_conversation(examples):
    # Join the list into a single string if it's a list of sentences
    joined_conversations = [" ".join(conv) if isinstance(conv, list) else conv for conv in examples['conversations']]
    
    # Tokenize the joined conversations
    return tokenizer(joined_conversations, truncation=True, max_length=512, padding="max_length", return_tensors="pt")

# Tokenize the dataset
tokenized_dataset = dataset.map(format_conversation, batched=True)

# Remove any columns not needed for training (e.g., original text fields)
tokenized_dataset = tokenized_dataset.remove_columns(["conversations", "source"])

# Ensure the format is PyTorch-friendly
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


In [5]:
# Low-Rank Adaptation (LoRA) configuration for efficient fine-tuning
peft_config = LoraConfig(
    lora_alpha=16,             # Scaling factor for LoRA updates
    lora_dropout=0.05,         # Dropout rate applied to LoRA layers
    r=16,                      # Rank of the LoRA decomposition
    bias="none",               # No bias is added to the LoRA layers
    task_type="CAUSAL_LM",     # Specify the task as causal language modeling
    target_modules=[           # Modules to apply LoRA to
        'k_proj', 'q_proj', 'v_proj', 'o_proj',
        'gate_proj', 'down_proj', 'up_proj'
    ]
)

In [6]:
# Define training arguments for the fine-tuning process
training_arguments = TrainingArguments(
    output_dir="./v2_mistral7b_results",  # Directory for saving model checkpoints and logs
    eval_strategy="steps",                # Evaluation strategy: evaluate every few steps
    do_eval=True,                         # Enable evaluation during training
    optim="paged_adamw_8bit",             # Use 8-bit AdamW optimizer for memory efficiency
    per_device_train_batch_size=4,        # Batch size per device during training
    gradient_accumulation_steps=2,        # Accumulate gradients over multiple steps
    per_device_eval_batch_size=2,         # Batch size per device during evaluation
    log_level="debug",                    # Set logging level to debug for detailed logs
    logging_steps=10,                     # Log metrics every 10 steps
    learning_rate=1e-4,                   # Initial learning rate
    eval_steps=25,                        # Evaluate the model every 25 steps
    max_steps=100,                        # Total number of training steps
    save_steps=25,                        # Save checkpoints every 25 steps
    warmup_steps=25,                      # Number of warmup steps for learning rate scheduler
    lr_scheduler_type="linear",           # Use a linear learning rate scheduler
)

In [ ]:
# Initialize the Supervised Fine-Tuning (SFT) Trainer
trainer = SFTTrainer(
    model=model,                          # The pre-trained and prepared model
    train_dataset=tokenized_dataset['train'],  # Training dataset
    eval_dataset=tokenized_dataset['test'],    # Evaluation dataset
    peft_config=peft_config,              # LoRA configuration for efficient fine-tuning
    max_seq_length=512,                   # Maximum sequence length for inputs
    tokenizer=tokenizer,                  # Tokenizer for encoding the data
    args=training_arguments,              # Training arguments defined earlier
)

# Start the fine-tuning process
trainer.train()

## Performance Benchmark 


In [29]:

accuracy_score = load("accuracy")

class PerformanceBenchmark:
    """
    A class to benchmark the performance of a model on a given dataset.
    
    Attributes:
    -----------
    model : transformers.PreTrainedModel
        The model to be benchmarked.
    tokenizer : transformers.PreTrainedTokenizer
        The tokenizer associated with the model.
    dataset : datasets.Dataset
        The dataset on which the model's performance will be evaluated.
    """
    
    def __init__(self, model, tokenizer, dataset):
        """
        Initializes the PerformanceBenchmark with the provided model, tokenizer, and dataset.
        
        Parameters:
        -----------
        model : transformers.PreTrainedModel
            The model to be benchmarked.
        tokenizer : transformers.PreTrainedTokenizer
            The tokenizer for encoding the inputs for the model.
        dataset : datasets.Dataset
            The dataset on which the model's performance will be evaluated.
        """
        self.model = model
        self.tokenizer = tokenizer
        self.dataset = dataset

    def compute_parameters(self):
        """
        Computes the total number of parameters and the number of trainable parameters.
        
        Returns:
        --------
        dict :
            A dictionary containing:
            - `total_params`: The total number of parameters in the model.
            - `trainable_params`: The number of trainable parameters in the model.
        """
        total_params = sum(p.numel() for p in self.model.parameters())  # Total parameters
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)  # Trainable parameters
        
        return {
            "total_params": total_params,
            "trainable_params": trainable_params
        }

    def compute_size(self):
        """
        Computes the size of the model in terms of the number of parameters 
        and memory usage in megabytes (MB).

        Returns:
        --------
        dict :
            A dictionary containing the number of parameters (`num_params`) and 
            the model size in MB (`model_size_mb`).
        """
        num_params = sum(p.numel() for p in self.model.parameters())
        model_size_mb = sum(p.element_size() * p.nelement() for p in self.model.parameters()) / (1024**2)
        
        return {"num_params": num_params, "model_size_mb": model_size_mb}

    def time_pipeline(self):
        """
        Measures the total time and average time taken by the model to process 
        the dataset.
        
        This method will use the tokenizer to encode the inputs before passing them 
        to the model.

        Returns:
        --------
        dict :
            A dictionary containing the total processing time in seconds (`total_time_sec`) 
            and the average time per example (`avg_time_per_example_sec`).
        """
        start_time = time.time()
        
        for example in self.dataset:
            inputs = example['conversations']
            # Tokenize the input
            tokenized_input = self.tokenizer(inputs, return_tensors="pt").to(self.model.device)
            _ = self.model.generate(**tokenized_input, max_new_tokens=10)
        
        end_time = time.time()
        total_time = end_time - start_time
        avg_time_per_example = total_time / len(self.dataset) if len(self.dataset) > 0 else float('inf')
        
        return {"total_time_sec": total_time, "avg_time_per_example_sec": avg_time_per_example}

    def compute_latency(self):
        """
        Computes the average latency of the model, defined as the time taken 
        to process a single example from the dataset.

        Returns:
        --------
        dict :
            A dictionary containing the average latency in seconds (`avg_latency_sec`).
        """
        latencies = []
        
        for example in self.dataset:
            inputs = example['conversations']
            # Tokenize the input
            tokenized_input = self.tokenizer(inputs, return_tensors="pt").to(self.model.device)
            
            start_time = time.time()
            _ = self.model.generate(**tokenized_input, max_new_tokens=10)
            end_time = time.time()
            
            latencies.append(end_time - start_time)
        
        avg_latency = sum(latencies) / len(latencies) if len(latencies) > 0 else float('inf')
        return {"avg_latency_sec": avg_latency}

    def compute_throughput(self):
        """
        Computes the throughput of the model, defined as the number of examples 
        processed per second.

        Returns:
        --------
        dict :
            A dictionary containing the throughput in examples per second (`throughput_examples_per_sec`).
        """
        start_time = time.time()
        
        for example in self.dataset:
            inputs = example['conversations']
            # Tokenize the input
            tokenized_input = self.tokenizer(inputs, return_tensors="pt").to(self.model.device)
            _ = self.model.generate(**tokenized_input, max_new_tokens=10)
        
        end_time = time.time()
        total_time = end_time - start_time
        throughput = len(self.dataset) / total_time if total_time > 0 else 0
        
        return {"throughput_examples_per_sec": throughput}
    

    def run_benchmark(self):
        """
        Runs all the benchmark metrics (size, time, latency, throughput, and FLOPs) 
        and returns the results.

        Returns:
        --------
        dict :
            A dictionary containing all the computed metrics for the model. 
            Includes size, parameters, time, latency, throughput, and FLOPs estimates.
        """
        metrics = {}
        metrics['Size'] = self.compute_size()
        metrics['Parameters'] = self.compute_parameters()
        metrics['Time'] = self.time_pipeline()
        metrics['Latency'] = self.compute_latency()
        metrics['Throughput'] = self.compute_throughput()
        return metrics
    
# Instantiate the PerformanceBenchmark class with the model, tokenizer, and test dataset
benchmark = PerformanceBenchmark(model, tokenizer, dataset['test'])

# Run the benchmark to compute performance metrics
results = benchmark.run_benchmark()

# Display the benchmark results
print(results)